In [9]:
# from collections import Counter, defaultdict
from itertools import chain, combinations
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import pandas as pd
from scipy.sparse import csr_matrix
import string
import nltk
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

##################################################################################################
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~This function tokenizes the text~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
##################################################################################################

def tokenize(docs, keep_internal_punct=False):
    doc=str(docs)
    l= doc.lower()
    l1=[]
    if  not keep_internal_punct :
        l1=re.sub("\W+",' ',l).split()
    else:
        for x in l.split():
            l1.append(x.strip(string.punctuation))
    return np.array([d.lower() for d in l1 if len(d)>=3])

##################################################################################################
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~This function creates a csr matrix~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
##################################################################################################
def create_csv(vocab,x,qr):
    global wordlist
    r=list()
    c=list()
    d=list()
    i=0
    pt = "newJ"
    for l in x:
        wordlist=nltk.FreqDist(l)
        for a,b in wordlist.items():
            if a in vocab.keys():
                d.append(b)
                r.append(i)
                c.append(vocab[a])
        i=i+1
    #print(wordlist)
    val = len(vocab.keys())
    data=np.array(d)
    row=np.array(r)
    col=np.array(c)
    return csr_matrix((data,(row,col)),shape=(i,val))

############################################################################################################
#~~~~~~~This function classify a tweet as a positive, negative or neutral using sentiment analysis~~~~~~~~~#
############################################################################################################
def classify():
    data=['Chicago-classify.csv','London-classify.csv','Sydney-classify.csv']
    city_name = ['Chicago', 'London','Sydney']
    x=list()
    y=list()
    qr = 0
    labels=list()
    
    file1 = open('classify.txt', 'w+')
    ps=pd.read_csv("pos.csv",sep='\t')
    tweets=ps['tweets'].tolist()
    twee = 0
    for t in tweets:
        l=list(tokenize(t))
        x.extend(l)
    for t in tweets:
        l=list(tokenize(t))
        y.append(l)
        twee = twee+1
    for i in range(twee):
        labels.append(1)

    ns=pd.read_csv("neg.csv",sep='\t')
    
    tweetsn=ns['tweets'].tolist()
    pwee = 0
    for t in tweetsn:
        l=list(tokenize(t))
        x.extend(l)
    for t in tweetsn:
        l=list(tokenize(t))
        y.append(l)
        pwee = pwee + 1
    for i in range(pwee):
        labels.append(0)
    
    wordlist=nltk.FreqDist(x)
    vocab=dict()
    min_freq=2
    i=0
    
    for key,values in wordlist.items():
        if values>=min_freq:
            vocab[key]=i
            i+=1
    X=create_csv(vocab,y,qr)
    clf=LogisticRegression()
    clf.fit(X.toarray(),labels)
    posv=0
    negv=0
    flag=0
    for name in data:
        y=[]
        ns=pd.read_csv(name,sep='\t')
        tweetsn=ns['text'].tolist()
        for t in tweetsn:
            l=list(tokenize(t))
            y.append(l)
        
        X=create_csv(vocab,y,qr)
        lb=clf.predict(X.toarray())
        lab=list(lb)
        at = str(len(lb))
        print("For State "+name+" total number of tweets: "+at)
        positive_count = str(lab.count(1))
        negative_count = str(lab.count(0)) 
        print("Total positive tweets:"+positive_count)
        print("Total negative tweets:"+negative_count)
        if positive_count > negative_count:
            print("More positive tweets i.e. "+ positive_count + " in the city of " + name)
        else:
            print("More negative tweets in the city of "+name)
        
        
        count_one = lab.count(1)
        count_zero = lab.count(0)
        posv=posv+count_one
        negv=negv+count_zero
        pt=tweetsn[lab.index(1)]
        nt=tweetsn[lab.index(0)]
    if file1.mode == 'w+':
        file1.write("Total number of positive tweets:"+str(posv)+"\nExample of a positive tweet is \n"+pt)
        file1.write("\nTotal number of negative tweets:"+str(negv)+"\nExample of a negative tweet is \n"+nt)
        file1.close()
    

def main():
    """ Main method. You should not modify this. """
    newt = 0
    classify()
    
if __name__ == '__main__':
    main()

For State Chicago-classify.csv total number of tweets: 100
Total positive tweets:46
Total negative tweets:54
More negative tweets in the city of Chicago-classify.csv
For State London-classify.csv total number of tweets: 100
Total positive tweets:23
Total negative tweets:77
More negative tweets in the city of London-classify.csv
For State Sydney-classify.csv total number of tweets: 100
Total positive tweets:47
Total negative tweets:53
More negative tweets in the city of Sydney-classify.csv
